In [127]:
from sklearn.datasets import load_digits
import pandas as pd
import numpy as np
from sklearn.metrics import confusion_matrix
from sklearn import metrics

In [13]:
digits = load_digits()
train_x = digits['data']
train_y = digits['target']

### k-means clustering implemented in problem1

- I need to change part of my code in problem1 so it can fit 64 demensional data
- still use eulucidian as poximity measure, though it may cause problem

In [65]:
class KMeans(object):
    
    '''
        simple kmeans impletation:
        n_clusters : number of clusters to find 
        max_iter : max iter algorithum will run even no converge
        tolerance : when change of center less than it, treat this as converge

        init : basic config
        fit: read and fit training data

        input data : the dataset should be a 2d numpy matrix, dataset[:,0] is x, the other is y
    '''
    
    def __init__(self, n_clusters):
        
        self.n_clusters = n_clusters
        #self.max_iter = max_iter
        #self.tol = tol
    
    def fit(self, dataset):
        
        # random choose center based on n_clusters
        self.centers = dataset[np.random.choice(dataset.shape[0]
                                               ,self.n_clusters,replace = False),:]
        self.dataset = dataset
        self.dem = dataset.shape[1]
        old_centers = np.copy(self.centers)
        self.update_leader()
        
        while old_centers.all() != self.centers.all():
            self.update_leader()
        
        print "finish"
            

    # find each node should belong to which cluster    
    def update_leader(self):
        
        # update the leader and store them in dataset last col
        new_leaders = np.apply_along_axis(self.find_leader, 1, self.dataset)
        if self.dataset.shape[1] == self.dem:
            new_leaders = np.expand_dims(new_leaders, axis=1)
            self.dataset = np.hstack((self.dataset, new_leaders))
        else:
            self.dataset = np.insert(self.dataset, self.dem + 1 , new_leaders ,axis =1)
        
        # find new center: use simple mean as new center
        for center in range(len(self.centers)):
            for cenetr_dem in range(self.dem):
                single_center = []
                single_center.append(np.mean(self.dataset[self.dataset[:,cenetr_dem + 1] == center,cenetr_dem]))
            self.centers[center] = single_center
            
            
    # the center number begin from 0    
    def find_leader(self, node):
        leader = float('NaN')
        min_dis = float('Inf')
        
        '''
            This dict store each demional of the cluster 
        '''
        center_dems = {}
        for center_dem in range(self.dem):
            center_dems[center_dem] = self.centers[:,center_dem]
         
        eu_dis = 0
        
        for i in range(self.n_clusters) :
            for center_dem in range(self.dem):
                eu_dis += np.square(node[center_dem] - center_dems[center_dem][i])
            
            eu_dis = np.sqrt(eu_dis)
            if eu_dis < min_dis :
                min_dis = eu_dis
                leader = i
                
        return np.int16(leader)
    
    def get_data(self):
        return self.dataset
    
    def get_center_num(self):
        return len(self.center)

In [152]:
kmeans = KMeans(n_clusters = 10)
kmeans.fit(train_x)

finish


#### Kmeans: do majority vote

In [153]:
def trans_cluster(pre_cluster,true_y):
    vote_df = pd.DataFrame(pre_cluster, columns = ['pre_cluster'])
    vote_df['true_cluster'] = true_y
    vote_df['trans_cluster'] = 0
    
    for c_name in range(10):
        c_name_group = vote_df.loc[vote_df['pre_cluster'] == c_name]
        trans_name = c_name_group['true_cluster'].value_counts().idxmax(axis = 0)
        vote_df.loc[vote_df['pre_cluster'] == c_name,'trans_cluster'] = int(trans_name)
        
    return vote_df

In [154]:
kmeans_vote = trans_cluster(kmeans.get_data()[:,-1],train_y)
kmeans_vote.head()

,pre_cluster,true_cluster,trans_cluster
0,7.0,0,0
1,2.0,1,1
2,3.0,2,8
3,1.0,3,3
4,4.0,4,4


#### kmeans: confusion matrix

In [155]:
confusion_matrix(kmeans_vote['true_cluster'],kmeans_vote['trans_cluster'])

array([[171,   1,   0,   0,   1,   0,   0,   0,   5,   0],
       [  0,  56,   0,   0,  56,   0,   0,  10,  60,   0],
       [ 11,  16,  79,   4,   0,   1,   0,  54,  12,   0],
       [  2,  12,  47, 106,   0,   2,   0,   4,  10,   0],
       [  1,   0,   0,   0, 145,   0,   0,  24,  11,   0],
       [  0,  19,   2,   0,  25, 114,   0,  19,   3,   0],
       [  9,   0,   0,   0, 101,   7,   0,   0,  64,   0],
       [  0,  20,   1,   0,   1,   0,   0, 153,   4,   0],
       [  0,  43,   2,   2,   0,   7,   0,   7, 113,   0],
       [ 12,  29,  48,  37,   0,  32,   0,   7,  15,   0]])

#### kmeans: Fowlkes and Mallows index evaluation

In [156]:
metrics.fowlkes_mallows_score(kmeans_vote['true_cluster'],kmeans_vote['trans_cluster'])

0.41765609242192703

### Agglomerative clustering with Ward linkage

In [129]:
from sklearn.cluster import AgglomerativeClustering

In [146]:
agg_cluster = AgglomerativeClustering(linkage='ward',
                                affinity= "euclidean",
                                n_clusters= 10)
precluster = agg_cluster.fit_predict(train_x)

#### Agglomerative clustering : majority vote

In [149]:
agg_vote_df = trans_cluster(precluster,train_y)
agg_vote_df.head()

,pre_cluster,true_cluster,trans_cluster
0,7,0,0
1,9,1,1
2,4,2,8
3,1,3,3
4,5,4,4


#### Agglomerative clustering : confusion matrix

In [157]:
confusion_matrix(agg_vote_df['true_cluster'],agg_vote_df['trans_cluster'])

array([[178,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [  0, 155,  27,   0,   0,   0,   0,   0,   0,   0],
       [  0,   0, 166,   0,   0,   0,   0,   1,  10,   0],
       [  0,   0,   0, 169,   0,   0,   0,   1,  13,   0],
       [  0,   0,   0,   0, 178,   0,   0,   3,   0,   0],
       [  0,   0,   0,   2,   0, 179,   1,   0,   0,   0],
       [  0,   0,   0,   0,   0,   0, 180,   0,   1,   0],
       [  0,   0,   0,   0,   0,   0,   0, 179,   0,   0],
       [  0,   3,   4,   1,   0,   0,   0,   1, 165,   0],
       [  0,  20,   0, 145,   0,   2,   0,  11,   2,   0]])

#### Agglomerative clustering : Fowlkes and Mallows index evaluation

In [158]:
metrics.fowlkes_mallows_score(agg_vote_df['true_cluster'],agg_vote_df['trans_cluster'])

0.83213950467054931

### AffinityPropagation

In [159]:
from sklearn.cluster import AffinityPropagation

In [184]:
af = AffinityPropagation().fit(train_x)

#### AffinityPropagation clustering : majority vote

In [185]:
affi_vote = trans_cluster(af.labels_,train_y) 
affi_vote.head()

,pre_cluster,true_cluster,trans_cluster
0,76,0,0
1,66,1,0
2,2,2,2
3,85,3,0
4,102,4,0


#### AffinityPropagation clustering : confusion matrix

In [186]:
confusion_matrix(affi_vote['true_cluster'],affi_vote['trans_cluster'])

array([[178,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [175,   7,   0,   0,   0,   0,   0,   0,   0,   0],
       [167,   0,   9,   1,   0,   0,   0,   0,   0,   0],
       [153,   0,   0,  29,   0,   1,   0,   0,   0,   0],
       [181,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [154,   0,   0,   0,   0,  28,   0,   0,   0,   0],
       [159,   0,   0,   0,   0,   0,  22,   0,   0,   0],
       [153,   0,   0,   0,   0,   0,   0,  26,   0,   0],
       [174,   0,   0,   0,   0,   0,   0,   0,   0,   0],
       [179,   0,   0,   0,   0,   1,   0,   0,   0,   0]])

#### AffinityPropagation clustering : Fowlkes and Mallows index evaluation

In [187]:
metrics.fowlkes_mallows_score(affi_vote['true_cluster'],affi_vote['trans_cluster'])

0.29757530338611876